Read More
- https://dacon.io/en/competitions/official/236127/codeshare/8791
- https://www.kaggle.com/code/shivanshuman/molecular-fingerprints
- https://deepchem.io/tutorials/the-basic-tools-of-the-deep-life-sciences/
- https://library.uos.ac.kr/search/detail/CAT000000682728

In [ ]:
target, seed = 'IC50_nM', 42

!pip install torch dgl rdkit pandas scikit-learn

Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 66.2 MB/s eta 0:00:00
Using cached nvidia_nvjitlink_cu12-12.6.20-py3-none-manylinux2014_x86_64.whl (19.7 MB)


In [ ]:
!pip install dgl deepchem

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [ ]:
import dgl
from dgl import DGLGraph
from dgl.nn.pytorch import GraphConv

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/usr/local/lib/python3.10/dist-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


FileNotFoundError: Cannot find DGL C++ graphbolt library at /usr/local/lib/python3.10/dist-packages/dgl/graphbolt/libgraphbolt_pytorch_2.3.1.so

In [ ]:
train = pd.read_csv('/content/train.csv')[['Smiles', 'IC50_nM']]
train.head()

NameError: name 'pd' is not defined

In [ ]:
# SMILES을 문자열로 변환하는 함
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    g = dgl.DGLGraph()
    num_atoms = mol.GetNumAtoms()
    g.add_nodes(num_atoms)

    for i in range(num_atoms):
        atom = mol.GetAtomWithIdx(i)
        g.ndata['h'] = torch.tensor([atom.GetAtomicNum()])

    for bond in mol.GetBonds():
        u = bond.GetBeginAtomIdx()
        v = bond.GetEndAtomIdx()
        g.add_edges(u, v)
        g.add_edges(v, u)

    return g

In [ ]:
smiles_to_graph(train['Smiles'])

TypeError: No registered converter was able to produce a C++ rvalue of type std::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> > from this Python object of type Series

In [ ]:
# 'SMILES', 'plc_50'의 DataFrame을 불러와
class MoleculeDataset(Dataset):
    def __init__(self, df):
        self.graphs = []
        self.labels = []

        for i in range(len(df)):
            graph = smiles_to_graph(df.iloc[i]['SMILES'])
            if graph is not None:
                self.graphs.append(graph)
                self.labels.append(df.iloc[i]['plc_50'])

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx], torch.tensor([self.labels[idx]], dtype=torch.float32)